In [1]:
!gdown 15R1tP_ouELsObDb9M4RTq4jRQ4xHWj07

Downloading...
From: https://drive.google.com/uc?id=15R1tP_ouELsObDb9M4RTq4jRQ4xHWj07
To: /content/public_data_dev.zip
100% 6.46M/6.46M [00:00<00:00, 31.9MB/s]


In [1]:
!rm -rf public_data emo_unicode.py
!gdown 1UNFaBN1JIg0rSdv9RYXr4jZm-F7-XSi8
!unzip -q public_data_dev.zip

'rm' is not recognized as an internal or external command,
operable program or batch file.
Downloading...
From (original): https://drive.google.com/uc?id=1UNFaBN1JIg0rSdv9RYXr4jZm-F7-XSi8
From (redirected): https://drive.google.com/uc?id=1UNFaBN1JIg0rSdv9RYXr4jZm-F7-XSi8&confirm=t&uuid=04cccc86-bd12-4b3e-ab60-9189e91499e6
To: e:\eLearning\code\TermArshad1\SemEval2025_Task11\output\new 2025\temp\emo_unicode.py

  0%|          | 0.00/304k [00:00<?, ?B/s]
100%|██████████| 304k/304k [00:00<00:00, 861kB/s]
100%|██████████| 304k/304k [00:00<00:00, 856kB/s]
unzip:  cannot find either public_data_dev.zip or public_data_dev.zip.zip.


INSTALLED

In [4]:
!pip install -q clean-text[gpl]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.1 MB/s eta 0:00:00


In [253]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import pandas as pd
import time
import os
from sklearn import metrics
import nltk
from nltk.tokenize import word_tokenize
from emo_unicode import UNICODE_EMOJI
from tqdm.notebook import tqdm
from tqdm import tqdm
from cleantext import clean

In [254]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sadegh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**load and preprocess**



constant


In [255]:
emoji_regex = r"[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\u2600-\u26FF\u2700-\u27BF]"
labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
kinds=["train","dev"]
tasks=["track_a","track_b","track_c"]
langs=["afr","amh","arq","ary","chn","deu","eng","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]



In [267]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text,lang="en"):
    text = text.strip()
    text = re.sub(r"&.*?;",'',text)

    # cleaning htmls
    text = cleanhtml(text)
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=True,
        # replace_with_url="",
        # replace_with_email="",
        # replace_with_phone_number="",
        # replace_with_number="",
        # replace_with_digit="0",
        # replace_with_currency_symbol="",
        lang=lang[:2]
    )

    # normalizing
    text = str(text).lower()

    
    # Base pattern without the conditional part
    base_pattern = (
        # u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # symbols & pictographs
        u"\U0001F680-\U0001F6FF" # transport & map symbols
        u"\U0001F1E0-\U0001F1FF" # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
    )

    # Add the conditional part based on the value of `chn`
    if lang!="chn":
        base_pattern += u"\U000024C2-\U0001F251"

    # Compile the regex pattern
    wierd_pattern = re.compile(f"[{base_pattern}]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)

    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

def convert_emoticons(text):
  emojis=re.findall(emoji_regex,text)
  # return emojis
  for emot in emojis:
    if emot not in UNICODE_EMOJI:
      # print("gosh")
      text = re.sub(u'('+emot+')',"", text)
    else:
      text = re.sub(u'('+emot+')', '('+UNICODE_EMOJI[emot].replace('_',' ').replace(':','')+') ', text)
  return text


In [268]:
a=convert_emoticons("2018 y lo sigo escuchando Like si tú también , te amo @<username> amor terricola 🖖❤️")
a

'2018 y lo sigo escuchando Like si tú también , te amo @<username> amor terricola (vulcan salute) (red heart) ️'

In [269]:
cleaning(a,lang="esp")

'2018 y lo sigo escuchando like si tú también te amo amor terricola vulcan salute red heart '

In [270]:
def safe_tokenize(message):
    if isinstance(message, str):
        return word_tokenize(message)
    else:
        return []

In [271]:
tqdm.pandas()
for kind in kinds:
  for task in tasks:
    for lang in langs:
      path=f"public_data_dev/{task}/{kind}/{lang}.csv"
      # if kind=="dev":
      #   path=path.replace(lang,lang+task[-2:])
      if not os.path.isfile(path):
        print("not found:",path)
        continue
      train_data=pd.read_csv(path)
      train_data["lang"]=lang
      train_data['clean_message']=train_data['text'].progress_apply(convert_emoticons).progress_apply(cleaning,args=(lang,))
      train_data = train_data.reset_index(drop=True)
      train_data=train_data.drop(columns={"text"})
      try:
        train_data["disgust"]
        # print(f"{kind} {task} {lang} has column disgust")
      except:
        print(f"{kind} {task} {lang} doesn't have column disgust")

      processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
      directory = os.path.dirname(processed_path)
      if not os.path.exists(directory):
        os.makedirs(directory)

      train_data.to_csv(processed_path,index=False)



100%|██████████| 2768/2768 [00:00<00:00, 13419.35it/s]


train track_a eng doesn't have column disgust


100%|██████████| 2992/2992 [00:00<00:00, 7785.42it/s]


not found: public_data_dev/track_a/train/ind.csv
not found: public_data_dev/track_a/train/jav.csv
not found: public_data_dev/track_a/train/xho.csv
not found: public_data_dev/track_a/train/zul.csv
not found: public_data_dev/track_b/train/afr.csv


100%|██████████| 901/901 [00:00<00:00, 9307.04it/s]


...

100%|██████████| 249/249 [00:00<00:00, 12337.21it/s]


not found: public_data_dev/track_b/dev/vmw.csv
not found: public_data_dev/track_b/dev/yor.csv
not found: public_data_dev/track_b/dev/ind.csv
not found: public_data_dev/track_b/dev/jav.csv
not found: public_data_dev/track_b/dev/xho.csv
not found: public_data_dev/track_b/dev/zul.csv


100%|██████████| 116/116 [00:00<00:00, 12652.83it/s]


dev track_c eng doesn't have column disgust


100%|██████████| 875/875 [00:00<00:00, 10514.45it/s]


In [272]:
!zip -qr processed_data.zip processed_data

In [273]:
# train_data[labels].sum()

In [274]:
# data_counts=train_data[labels].value_counts()
# data_counts

data visualization


In [275]:
# train_data['num_words']=train_data['clean_message'].str.len()
# fig=plt.figure(figsize=(12,8))
# sns.set_style('dark')

# sns.displot(train_data['num_words'], kde=True,color='Blue')
# mean_val = train_data['num_words'].mean()
# print(f'\nMean:{mean_val:.2f}')
# plt.legend()
# plt.title('number of words in each message',fontsize=16,fontweight='bold')
# plt.xlabel('Number of Words', fontsize=12)
# plt.ylabel('Frequency', fontsize=12)

# plt.grid(True, linestyle='--', alpha=1)
# plt.show()

In [276]:
# big_len_data= train_data[train_data['clean_message'].apply(lambda x: len(x) >350)]
# big_len_data

In [277]:
# normal_len_data= train_data[train_data['clean_message'].apply(lambda x: len(x)<=350)]
# normal_len_data